In [3]:
import json
from typing import Generator, Dict, Any
from pathlib import Path
from tqdm import tqdm
from dotenv import load_dotenv
import sys

root_path = Path.cwd().parent.parent.parent
root_path = root_path
print(f"{root_path=}")
sys.path.append(str(root_path))
load_dotenv(dotenv_path=root_path / "src" / "config" / ".env")


root_path=PosixPath('/mnt/storage_2/scratch/pl0145-01/jsmok/qa/mt')


True

In [11]:
# Generator function to read and process a JSONL file line by line
def read_jsonl_file(file_path: Path) -> Generator[Dict[str, Any], None, None]:
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Parse the JSON object from each line
            json_obj = json.loads(line.strip())
            yield json_obj

# Example usage
passages_path = root_path / "data" / "ipipan-polqa"/ "passages.jsonl"
for json_obj in read_jsonl_file(passages_path):
    print(json_obj)
    break

{'id': '2-0', 'title': 'AWK', 'text': 'AWK – interpretowany język programowania, którego główną funkcją jest wyszukiwanie i przetwarzanie wzorców w plikach lub strumieniach danych. Jest także nazwą programu początkowo dostępnego dla systemów operacyjnych będących pochodnymi UNIX-a, obecnie także na inne platformy. AWK jest językiem, który w znacznym stopniu wykorzystuje tablice asocjacyjne, stringi i wyrażenia regularne.'}


In [3]:
from src.config.configs import get_config
config = get_config("config1")

In [20]:
from src.lib.VectorDb.tools import get_vector_db
vector_db = get_vector_db(config)

In [4]:
from src.lib.Embeddings.tools import get_embedding_model
embeddings = get_embedding_model(config)

2024-06-12 17:54:29,509 - INFO - device: cuda
/mnt/storage_2/scratch/pl0145-01/jsmok/qa/mt/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/mnt/storage_2/scratch/pl0145-01/jsmok/qa/mt/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-12 17:54:32,398 - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/mnt/storage_2/scratch/pl0145-01/jsmok/qa/mt/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [31]:
def process_jsonl(file_path: str) -> tuple[tuple[dict[str, str]], tuple[str], tuple[int]]:
    id_title_list = []
    text_list = []
    len_list = []
    
    num_lines = sum(1 for _ in open(file_path, 'r', encoding='utf-8'))
    i=0
    for json_obj in tqdm(read_jsonl_file(file_path), total=num_lines, desc="Processing JSONL"):
        if 'id' in json_obj and 'title' in json_obj and 'text' in json_obj:
            id_title_list.append({"id": json_obj['id'], "title": json_obj['title']})
            text_list.append(json_obj['text'])
            len_list.append(len(json_obj['text']))
        i+=1
        if i > 150:
            break

    combined_list = list(zip(id_title_list, text_list, len_list))
    sorted_combined_list = sorted(combined_list, key=lambda x: x[2])
    id_title_list, text_list, len_list = zip(*sorted_combined_list)
    
    
    return id_title_list, text_list, len_list

In [32]:
meta, docs, lens = process_jsonl(passages_path)

Processing JSONL:   0%|          | 150/7097288 [00:00<01:13, 97045.44it/s]


In [33]:
DATA_DIR = "data"
SPLIT_INTO_CHUNKS = True
MAX_CHUNK_SIZE = 512
CHUNK_OVERLAP = 100
TEXT_SPLITTER_SEPARATOR = ""
VECTOR_DB_PATH = "vector_db"

MODEL_ID = "hf-e5"
VECTOR_DB_ID = "faiss"

import abc
import glob
import json
import logging
from functools import lru_cache

import torch
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS


    # Read docs


# split into chunks (if needed)
if SPLIT_INTO_CHUNKS:
    text_splitter = CharacterTextSplitter(
        chunk_size=MAX_CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP, separator=TEXT_SPLITTER_SEPARATOR
    )
    docs_chunks = text_splitter.create_documents(docs, meta)
    docs = [doc.page_content for doc in docs_chunks]
    metadata = [doc.metadata for doc in docs_chunks]
print(f"Number of documents: {len(docs)}" )
print(f"Number of metadata: {len(meta)}" )


Number of documents: 151
Number of metadata: 151


In [34]:
# calculate embeddings
embedding_model = get_embedding_model(config)
embeddings = embedding_model.embed_documents(docs)
docs_embeddings_pairs = list(zip(docs, embeddings))


In [24]:
# store data in vector db
vector_db = get_vector_db(config)


In [35]:
vector_db.store_embeddings(docs_embeddings_pairs, metadata)

2024-06-12 23:38:44,062 - WARNING - `embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [26]:
len(vector_db.index_to_docstore_id)

AttributeError: 'FAISSVectorDb' object has no attribute 'index_to_docstore_id'

In [27]:
vector_db.vector_db_path

PosixPath('/mnt/storage_2/scratch/pl0145-01/jsmok/data/vector_databases/faiss_vector_db_config_1')

In [36]:
import faiss

# Load the FAISS index from the .faiss file
index = faiss.read_index("/mnt/storage_2/scratch/pl0145-01/jsmok/data/vector_databases/faiss_vector_db_config_1/index.faiss")

# Get the number of indexed vectors
num_vectors = index.ntotal
print(f"Number of indexed vectors: {num_vectors}")

Number of indexed vectors: 151


In [9]:
from src.lib.data_tools.read_file import process_jsonl


In [5]:
from src.config.Config import Config


In [6]:
from src.lib.common.tools import sort_docs_by_len


In [7]:
from src.lib.VectorDb.tools import get_vector_db


In [8]:
from src.lib.Embeddings.tools import get_embedding_model